<h1>NLP with ML - Project

In [162]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import*
import pandas as pd
import re
from functools import reduce 
from tensorflow.keras.preprocessing.text import Tokenizer
dataFrame=pd.read_csv('/content/data2.csv')

In [163]:
def add1_smoothing(df, words):
  cWith = 0
  while cWith < len(words):
    if words[cWith] == 'ile':
        del words[cWith]
    cWith = cWith + 1
  
  dataDuygu = []
  dataYorum = []
  dataDuyguSentence = []
  for i in range(len(df)):
    dataDuyguSentence.append(df['Duygu'][i])
    for j in df['Yorum'][i].split():
      dataYorum.append(j)
      dataDuygu.append(df['Duygu'][i])

  tokenizer = Tokenizer(num_words = 1000)
  tokenizer.fit_on_texts(df['Yorum'])
  word_index = tokenizer.word_index
  vocab = len(tokenizer.word_counts)
  
  numOfPositive = dataDuygu.count(1)
  numOfNegative = dataDuygu.count(0)
  
  pPositive={}
  pNegative={}
  keyDictPositive = {}
  keyDictNegative = {}
  for i in words:
    keyDictPositive.update({i:0})
    keyDictNegative.update({i:0})
    pPositive.update({i:1})
    pNegative.update({i:1})

  counter=0
  for i in dataYorum:
    for j in words:
      if (j in i) and dataDuygu[counter]==1:
        keyDictPositive.update({j:keyDictPositive[j]+1})
      if (j in i) and dataDuygu[counter]==0:
        keyDictNegative.update({j:keyDictNegative[j]+1})
    counter+=1
  
  for i in words:
    pPositive.update({i:(keyDictPositive[i]+1)/(vocab+numOfPositive)})
    pNegative.update({i:(keyDictNegative[i]+1)/(vocab+numOfNegative)})

  pNumOfPositive = dataDuyguSentence.count(1)
  pNumOfNegative = dataDuyguSentence.count(0)
  numOfSentenceTotal = pNumOfPositive + pNumOfNegative
  pTotalPositive = pNumOfPositive / numOfSentenceTotal
  pTotalNegative = pNumOfNegative / numOfSentenceTotal

  for i in words:
    pTotalPositive *= pPositive[i]
    pTotalNegative *= pNegative[i]
  
  if pTotalPositive > pTotalNegative:
    return 1
  else:
    return 0


In [164]:
def negation(df):
  data=df['Yorum']
  dataDuygu=df['Duygu']
  pattern = r'\b(?:değil|yok)'
  dataNegation = []
  for c in (range(len(data))):
    neg = []
    endPoints = []
    if re.search(r'[?.,*!:;]',data[c])==None:
      endPoints.append(len(data[c]))
    for match in (re.finditer('[?.,*!:;]', data[c])):
      endPoints.append(match.end())
    for i in range(len(endPoints)):
      if i==0:
        if len(re.findall(pattern,data[c][0:endPoints[i]]))!=0:
          sentenceArray = data[c][0:endPoints[i]].split()
          for j in range(len(sentenceArray)):
            if re.search(pattern, sentenceArray[j])==None:
              sentenceArray[j]='değil_'+sentenceArray[j]
            else:
              sentenceArray[j]=sentenceArray[j]
        else:
          sentenceArray = data[c][0:endPoints[i]].split()
        neg.append(" ".join(sentenceArray))
      elif i!=len(endPoints):
        if len(re.findall(pattern,data[c][endPoints[i-1]:endPoints[i]]))!=0:
          sentenceArray = data[c][endPoints[i-1]:endPoints[i]].split()
          for j in range(len(sentenceArray)):
            if re.search(pattern, sentenceArray[j])==None:
              sentenceArray[j]='değil_'+sentenceArray[j]
            else:
              sentenceArray[j]=sentenceArray[j]
        else:
          sentenceArray = data[c][endPoints[i-1]:endPoints[i]].split()
        neg.append(" ".join(sentenceArray))
        if len(data[c])!=endPoints[-1]:
          if len(re.findall(pattern,data[c][endPoints[-1]:len(data[c])]))!=0:
            sentenceArray = data[c][endPoints[-1]:len(data[c])].split()
            for j in range(len(sentenceArray)):
              if re.search(pattern, sentenceArray[j])==None:
                sentenceArray[j]='değil_'+sentenceArray[j]
              else:
                sentenceArray[j]=sentenceArray[j]
          else:
            sentenceArray = data[c][endPoints[-1]:len(data[c])].split()
          neg.append(" ".join(sentenceArray))
    dataNegation.append(" ".join(neg))
  frame=[]
  for i in range(len(dataNegation)):
    frame.append([dataNegation[i],dataDuygu[i]])
  dfNegation=pd.DataFrame(frame,columns = ['Yorum', 'Duygu'])
  return dfNegation



In [165]:
def lexicon(df):
  dataYorum = df["Yorum"]
  dataDuygu = df['Duygu']
  dataPositive = []
  dataNegative = []

  for i in range(len(dataYorum)):
    if dataDuygu[i] == 1:
      dataPositive.append(dataYorum[i])
    else:
      dataNegative.append(dataYorum[i])
  tokenizerPositive = Tokenizer(num_words = 1000)
  tokenizerPositive.fit_on_texts(dataPositive)
  word_index_positive = tokenizerPositive.word_counts
  tokenizerNegative = Tokenizer(num_words = 1000)
  tokenizerNegative.fit_on_texts(dataNegative)
  word_index_negative = tokenizerNegative.word_counts
  vocab = len(tokenizerPositive.word_counts)+len(tokenizerNegative.word_counts) 
  allPositiveCount=0
  allNegativeCount=0
  
  for key,value in word_index_positive.items():
    allPositiveCount+=value
  for key,value in word_index_negative.items():
    allNegativeCount+=value
  
  lexiconDuygu=[]
  lexiconYorumScore=[]
  lexiconYorum=[]
  lexiconPositive={}
  lexiconNegative={}
  for key,value in word_index_positive.items():
    lexiconYorum.append(key)
    lexiconYorumScore.append((value+1)/(allPositiveCount+vocab))
    lexiconDuygu.append(1)
    lexiconPositive.update({key:(value+1)/(allPositiveCount+vocab)})
  for key,value in word_index_negative.items():
    lexiconYorum.append(key)
    lexiconYorumScore.append((value+1)/(allNegativeCount+vocab))   
    lexiconDuygu.append(-1)
    lexiconNegative.update({key:(value+1)/(allNegativeCount+vocab)})

  frame=[]
  for i in range(len(lexiconYorum)):
    frame.append([lexiconYorum[i],lexiconYorumScore[i],lexiconDuygu[i]])
  lexicon=pd.DataFrame(frame,columns = ['Yorum',"Score", 'Duygu'])
  return lexicon,lexiconPositive,lexiconNegative

  

In [166]:
dataframe, y_Test = train_test_split(dataFrame,test_size=0.05,random_state=0)
y_pred = []
y_pred_negation = [] 
y_true = []
for i in y_Test['Duygu']:
  y_true.append(i)
for i in y_Test['Yorum']:
  y_pred.append(add1_smoothing(dataFrame, i.split()))
  y_pred_negation.append(add1_smoothing(negation(dataFrame),i.split()))

In [167]:
lex,lex_test = train_test_split(lexicon(dataFrame)[0],test_size=0.001,random_state=5)
lexP=lexicon(dataFrame)[1]
lexN=lexicon(dataFrame)[2]
# Positive or negative labels have been set manually.
testArray=['çok','iyi','orta','fiyat','tavsiye','telefon','kadar','kart','kamera','cihaz']
lex_pred = []
lex_true = [1,1,0,0,1,0,0,1,1,0]

for i in testArray:
  if lexP[i]>=lexN[i]:
    lex_pred.append(1)
  else:
    lex_pred.append(0)

In [169]:
print('Add1 Smoothing Scores')
print(accuracy_score(y_true, y_pred))
print(precision_score(y_true, y_pred, average='micro'))
print(recall_score(y_true, y_pred, average='micro'))
print(f1_score(y_true, y_pred, average='micro'))
print('Add1 Smoothing with Negation Scores')
print(accuracy_score(y_true, y_pred_negation))
print(precision_score(y_true, y_pred_negation, average='micro'))
print(recall_score(y_true, y_pred_negation, average='micro'))
print(f1_score(y_true, y_pred_negation, average='micro'))
print('Lexicon Scores')
print(accuracy_score(lex_true, lex_pred))
print(precision_score(lex_true, lex_pred, average='micro'))
print(recall_score(lex_true, lex_pred, average='micro'))
print(f1_score(lex_true, lex_pred, average='micro'))


Add1 Smoothing Scores
0.9591836734693877
0.9591836734693877
0.9591836734693877
0.9591836734693877
Add1 Smoothing with Negation Scores
0.9591836734693877
0.9591836734693877
0.9591836734693877
0.9591836734693877
Lexicon Scores
0.5
0.5
0.5
0.5
